<a href="https://colab.research.google.com/github/s1scottd/CIFAR-10_Image-Classification-CNN/blob/main/CIFAR_10_Image_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries. 

In [21]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from keras.datasets import mnist
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from keras.datasets import cifar10

##Prepare the data

Define a reshape and normalize function

In [22]:
def reshape_and_normalize(data):
  shape = data.shape
  data = np.reshape(data,(shape[0],32,32,3))
  data = data/255.
  return data

Load, Reshape and Normalize the CIFR-10 Dataset

In [23]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train[:-10000]
X_val = X_train[-10000:]
y_train = y_train[:-10000]
y_val = y_train[-10000:]

X_train = reshape_and_normalize(X_train)
X_test = reshape_and_normalize(X_test)
X_val = reshape_and_normalize(X_val)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")
print(f"X_val shape: {X_val.shape}")

X_train shape: (40000, 32, 32, 3)
X_test shape:  (10000, 32, 32, 3)
X_val shape: (10000, 32, 32, 3)


# Convolutional Neural Network without Tuning

Define and compile the CNN model

In [24]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10)
])

# Compile the model
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

Fit the CNN model

In [25]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
1250/1250 [==============================] - 8s 6ms/step - loss: 1.4960 - accuracy: 0.4604 - val_loss: 1.2229 - val_accuracy: 0.5669
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.1426 - accuracy: 0.5972 - val_loss: 1.0190 - val_accuracy: 0.6468
Epoch 3/10
1250/1250 [==============================] - 7s 5ms/step - loss: 1.0026 - accuracy: 0.6489 - val_loss: 0.9021 - val_accuracy: 0.6880
Epoch 4/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9152 - accuracy: 0.6811 - val_loss: 0.8696 - val_accuracy: 0.7001
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.8460 - accuracy: 0.7070 - val_loss: 0.7599 - val_accuracy: 0.7321
Epoch 6/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.7868 - accuracy: 0.7281 - val_loss: 0.6840 - val_accuracy: 0.7666
Epoch 7/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.7310 - accuracy: 0.7462 - val_loss: 0.7014 - val_accuracy:

Evaluate the model

In [26]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('Test accuracy:', test_acc)

313/313 - 1s - loss: 0.9310 - accuracy: 0.6944 - 669ms/epoch - 2ms/step
Test accuracy: 0.6944000124931335


# Convolutional Neural Network with Tuning

Install and import Tuner

In [27]:
!pip install keras-tuner --upgrade
import keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Create a tunable model

In [28]:


def build_model(hp):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(
        # tune number of units
        units = hp.Int('units', min_value=32, max_value=512, step=32),
        # tune the activation function to use
        activation=hp.Choice("activation", ["relu", "tanh"])))
  model.add(Dense(10, activation="softmax"))
  
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
  
  return model

Build and Compile the model

In [29]:
build_model(keras_tuner.HyperParameters())

Create the tuner

In [30]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='accuracy',
    max_trials=10,
    executions_per_trial=5)

Execute a search for the best model

In [31]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
best_model = tuner.get_best_models()[0]
print(f"Search Space Summary:\n{tuner.search_space_summary}")


Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
352               |352               |units
relu              |relu              |activation
0.002708          |0.00070828        |lr

Epoch 1/5
1250/1250 [==============================] - 14s 9ms/step - loss: 1.5149 - accuracy: 0.4496 - val_loss: 1.2332 - val_accuracy: 0.5553
Epoch 2/5
1250/1250 [==============================] - 8s 6ms/step - loss: 1.1834 - accuracy: 0.5840 - val_loss: 1.0507 - val_accuracy: 0.6361
Epoch 3/5
1238/1250 [============================>.] - ETA: 0s - loss: 1.0363 - accuracy: 0.6347

KeyboardInterrupt: ignored